In [3]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score

# 1. Load dataset
# Ganti path ini sesuai lokasi file CSV-mu
df = pd.read_excel("new_dataset.xlsx") 

# 2. Bersihkan nama kolom
df.columns = df.columns.str.strip()

# 3. Definisikan label asli
# Asumsinya kolom ke-9 (indeks 8) adalah label ground-truth
df['label_asli'] = df.iloc[:, 3]

# 4. Daftar fitur numerik yang akan diuji satu per satu
fitur_list = [
    'Kunjungan PerMinggu',
    'Waktu Rata-Rata Situs(menit)',
    'Produk Dilihat'
]

# 5. Bersihkan dan cast tiap kolom numerik menjadi float
for kol in fitur_list:
    df[kol] = (
        df[kol]
        .astype(str)
        .str.replace(',', '.', regex=False)
        .astype(float)
    )

# 6. Loop untuk menghitung ARI per fitur
results = []
n_clusters = df['label_asli'].nunique()

for feat in fitur_list:
    # 6a. Ambil kolom sebagai 2D array
    X_feat = df[[feat]].values
    
    # 6b. Standarisasi
    X_std = StandardScaler().fit_transform(X_feat)
    
    # 6c. Clustering dengan KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X_std)
    
    # 6d. Hitung ARI
    ari = adjusted_rand_score(df['label_asli'], kmeans.labels_)
    results.append((feat, ari))

# 7. Tampilkan urutan fitur berdasarkan ARI tertinggi
res_df = (
    pd.DataFrame(results, columns=['fitur', 'ARI'])
      .sort_values('ARI', ascending=False)
      .reset_index(drop=True)
)

print(res_df)


                          fitur       ARI
0  Waktu Rata-Rata Situs(menit)  0.038788
1           Kunjungan PerMinggu  0.033018
2                Produk Dilihat  0.031537
